<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/RadauDDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import jax.numpy as jnp
import jax
jax.config.update('jax_enable_x64', True)
from plotly.subplots import make_subplots
from scipy.integrate import Radau
from functools import partial

In [121]:
def dde_Radau(rhs, trange, y0, tau, rtol=1e-6, atol=1e-8, hist_N=int(10000)):
    t0,tf = trange
    neq= len(y0)

    def solve():

        ts=jnp.full(hist_N,jnp.inf)
        ts=ts.at[0].set(t0)
        ys=jnp.full((hist_N,neq),jnp.inf)
        ys=ys.at[0].set(y0)
        Qs=jnp.zeros((hist_N,neq,3))

        @jax.jit
        def yold(t, ts, ys, Qs):

            def before0(t):
                return y0

            def after0(t):
                i=jnp.searchsorted(ts,t, side='right')-1
                h=ts[i+1]-ts[i]
                x = (t-ts[i])/h
                p = x**np.arange(1,4)
                y= jnp.where(t<tf,Qs[i+1]@p+ys[i],ys[i])

                return y

            return jax.lax.cond(t<0,before0,after0, t)

        @jax.jit
        def rhs2(t,y, ts, ys, Qs):
            return rhs(t,y, lambda t: yold(t,ts,ys,Qs))

        jac=jax.jit(jax.jacobian(rhs2,1))
        solver=Radau(lambda t,y: rhs2(t,y, ts, ys, Qs), t0, y0, tf,
                     jac=lambda t,y: jac(t,y,ts,ys, Qs), rtol=1e-6,atol=1e-8)
        t=t0
        nstep=0
        while t<tf:
            solver.max_step=tau
            solver.step()
            nstep+=1
            t=solver.t
            ts=ts.at[nstep].set(solver.t)
            ys=ys.at[nstep].set(solver.y)
            Qs=Qs.at[nstep].set(solver.sol.Q)
        return jnp.vectorize(lambda t: yold(t, ts, ys, Qs), signature='()->(n)')

    return solve

In [122]:
def rhs(t,y, yold):
    return jnp.array([y[1], -y[0]+yold(t-0.1)[0]])

In [124]:
solver=dde_Radau(rhs, (0,10), jnp.array([1., 1.]), tau=0.1)

In [125]:
sol=solver()

In [126]:
tplot=jnp.linspace(0,10,11)
sol(tplot)

Array([[1.        , 1.        ],
       [1.95603391, 0.90895665],
       [2.82080613, 0.8220434 ],
       [3.60288996, 0.74344069],
       [4.31019196, 0.67235386],
       [4.94986266, 0.60806426],
       [5.52836889, 0.54992195],
       [6.0515591 , 0.49733913],
       [6.52472255, 0.44978421],
       [6.95264273, 0.40677643],
       [7.33964574, 0.367881  ]], dtype=float64)